In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession .\
builder .\
appName("spark") .\
config("spark.sql.warehouse.dir","/user/itv016843/warehouse") .\
enableHiveSupport() .\
master("yarn") .\
getOrCreate()

In [3]:
spark

In [4]:
from pyspark.sql.functions import *

In [5]:
from pyspark.sql import *

In [6]:
! hadoop fs -ls /public/trendytech/lendingclubproject/raw

Found 4 items
drwxr-xr-x   - itv005857 supergroup          0 2023-09-15 14:40 /public/trendytech/lendingclubproject/raw/customers_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-17 22:57 /public/trendytech/lendingclubproject/raw/loans_data_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:32 /public/trendytech/lendingclubproject/raw/loans_defaulters_csv
drwxr-xr-x   - itv005857 supergroup          0 2023-09-18 07:31 /public/trendytech/lendingclubproject/raw/loans_repayments_csv


In [7]:
df1 = spark.read \
.format("csv") \
.option("header",True) \
.load("/public/trendytech/lendingclubproject/raw/loans_data_csv")

In [8]:
df1.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title
56633077,b59d80da191f5b573...,3000.0,3000.0,36 months,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,36 months,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...
56473345,e5a140c0922b554b9...,20000.0,20000.0,36 months,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation
56463188,e12aefc548f750777...,11200.0,11200.0,60 months,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,60 months,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation


In [9]:
df2 = df1.withColumn("Time",current_timestamp())

In [10]:
df2.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
145499677,a703357afc7be3fe3...,10000.0,10000.0,36 months,8.19,314.25,Dec-2018,Fully Paid,debt_consolidation,Debt consolidation,2025-07-26 02:52:...
144538467,a0c637c3df6764663...,5000.0,5000.0,36 months,15.02,173.38,Dec-2018,Current,other,Other,2025-07-26 02:52:...
145515405,63571114d3a96e5bc...,7500.0,7500.0,36 months,10.33,243.17,Dec-2018,Current,debt_consolidation,Debt consolidation,2025-07-26 02:52:...
145207340,4db14234c3f2f87c1...,20400.0,20400.0,60 months,16.14,497.61,Dec-2018,Current,home_improvement,Home improvement,2025-07-26 02:52:...
145467050,88a6f97ff3afc51b6...,12000.0,12000.0,36 months,7.02,370.64,Dec-2018,Current,credit_card,Credit card refin...,2025-07-26 02:52:...


In [11]:
df2.count()

2260701

In [12]:
columns = [
    "loan_amnt", "funded_amnt", "term", "int_rate", "installment",
    "issue_d", "loan_status", "purpose", "title"
]

In [13]:
df3 = df2.na.drop(subset = columns)

In [14]:
df3.count()

2237341

In [15]:
df3.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
13048578,ae2a45f96691a33e2...,30000.0,30000.0,60 months,22.45,836.27,Mar-2014,Fully Paid,debt_consolidation,Debt consolidation,2025-07-26 02:52:...
13028587,bc56a941d1b4a0ded...,3500.0,3500.0,36 months,13.65,119.03,Mar-2014,Fully Paid,debt_consolidation,Debt consolidation,2025-07-26 02:52:...
13048646,ec235820c5f579fe6...,20000.0,20000.0,60 months,18.92,517.94,Apr-2014,Charged Off,medical,Medical expenses,2025-07-26 02:52:...
13045633,87c71f39b975e9b1c...,10000.0,10000.0,60 months,15.61,241.12,Mar-2014,Fully Paid,credit_card,Credit card refin...,2025-07-26 02:52:...
13018547,8ce13bae24094c318...,30000.0,30000.0,60 months,19.99,794.65,Mar-2014,Fully Paid,debt_consolidation,Debt consolidation,2025-07-26 02:52:...


In [16]:
df4 = df3.withColumn("term",regexp_replace(col("term"),"months"," ").cast("int")/12)

In [17]:
df4.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
56633077,b59d80da191f5b573...,3000.0,3000.0,3.0,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-26 02:52:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,3.0,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-26 02:52:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,3.0,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2025-07-26 02:52:...
56463188,e12aefc548f750777...,11200.0,11200.0,5.0,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2025-07-26 02:52:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,5.0,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-07-26 02:52:...


In [18]:
df5 = df4.withColumn("term",df4.term.cast("int"))

In [19]:
df5.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
56633077,b59d80da191f5b573...,3000.0,3000.0,3,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-26 02:52:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,3,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-26 02:52:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,3,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2025-07-26 02:52:...
56463188,e12aefc548f750777...,11200.0,11200.0,5,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2025-07-26 02:52:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,5,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-07-26 02:52:...


In [21]:
df5.groupBy("purpose").count().sort("count",ascending=False)

purpose,count
debt_consolidation,1263422
credit_card,511459
home_improvement,149141
other,138623
major_purchase,50050
medical,27247
small_business,24416
car,23815
vacation,15373
moving,15297


In [22]:
purpose_col = ["debt_consolidation", "credit_card", "home_improvement", "other", "major_purchase",
 "medical", "small_business", "car", "vacation", "moving", "house", "wedding",
 "renewable_energy", "educational"]

In [23]:
df6 = df5.withColumn("purpose",
                    when(col("purpose").isin(purpose_col),col("purpose")).otherwise("other")
                    )

In [26]:
df6.groupBy("purpose").count().sort("count",ascending=False)

purpose,count
debt_consolidation,1263422
credit_card,511459
home_improvement,149141
other,138876
major_purchase,50050
medical,27247
small_business,24416
car,23815
vacation,15373
moving,15297


In [27]:
df6.limit(5)

loan_id,member_id,loan_amnt,funded_amnt,term,int_rate,installment,issue_d,loan_status,purpose,title,Time
56633077,b59d80da191f5b573...,3000.0,3000.0,3,7.89,93.86,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-26 03:00:...
55927518,202d9f56ecb7c3bc9...,15600.0,15600.0,3,7.89,488.06,Aug-2015,Fully Paid,credit_card,Credit card refin...,2025-07-26 03:00:...
56473345,e5a140c0922b554b9...,20000.0,20000.0,3,9.17,637.58,Aug-2015,Fully Paid,debt_consolidation,Debt consolidation,2025-07-26 03:00:...
56463188,e12aefc548f750777...,11200.0,11200.0,5,21.99,309.27,Aug-2015,Fully Paid,home_improvement,Home improvement,2025-07-26 03:00:...
56473316,1b3a50d854fbbf97e...,16000.0,16000.0,5,20.99,432.77,Aug-2015,Charged Off,debt_consolidation,Debt consolidation,2025-07-26 03:00:...
